In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("/Users/saimanish/SeniorDesign/SDP/prototype/backend/SUMO/results/vehicle_data.csv")
df.head(30)

,Time,Vehicle_ID,Speed,Acceleration,Latitude,Longitude,Lane,Headway_Distance,Time_Gap,Speed_Limit,Lane_Change,From_Lane,To_Lane,Lane_Change_Reason,Collision
0,1.0,veh0,0.000000,0.000000,41.779108,-72.677119,17192247#1_1,inf,inf,22.22,False,NaN,NaN,NaN,False
1,2.0,veh0,1.377546,1.377546,41.779108,-72.677119,17192247#1_1,inf,inf,22.22,False,NaN,NaN,NaN,False
2,3.0,veh0,3.896292,2.518745,41.779108,-72.677119,17192247#1_1,inf,inf,22.22,False,NaN,NaN,NaN,False
3,4.0,veh0,6.245073,2.348782,41.779108,-72.677119,17192247#1_1,inf,inf,22.22,False,NaN,NaN,NaN,False
4,5.0,veh0,8.456776,2.211703,41.779108,-72.677119,17192247#1_1,inf,inf,22.22,False,NaN,NaN,NaN,False
5,6.0,veh0,9.824252,1.367475,41.779108,-72.677119,:8563990289_0_1,inf,inf,22.22,False,NaN,NaN,NaN,False
6,6.0,veh1,0.000000,0.000000,41.779105,-72.677098,-657499234#1_0,inf,inf,22.22,False,NaN,NaN,NaN,False
7,7.0,veh0,11.921781,2.097529,41.779108,-72.677119,:cluster_7224902542_7281703362_4_1,inf,inf,22.22,False,NaN,NaN,NaN,False
8,7.0,veh1,1.399670,1.399670,41.779105,-72.677098,-657499234#1_0,inf,inf,22.22,False,NaN,NaN,NaN,False
9,8.0,veh0,13.813484,1.891703,41.779108,-72.677119,17192247#3_1,inf,inf,22.22,False,NaN,NaN,NaN,False


In [8]:
# Speeding Violation
df['SpeedingViolation'] = df['Speed'] > df['Speed_Limit']

# Fast Acceleration (threshold = 2.5 m/s²)
df['FastAcceleration'] = df['Acceleration'] > 2.5

# Lane Change Frequency
df['LaneChangeFrequency'] = df.groupby('Vehicle_ID')['Lane_Change'].cumsum()

# Hard Braking (threshold = -2.5 m/s²)
df['HardBraking'] = df['Acceleration'] < -2.5

In [10]:
# Unsafe Headway Distance (threshold = 2 meters) - take into account speed
df['UnsafeHeadway'] = df['Headway_Distance'] < 2

# Unsafe Time Gap (threshold = 2 seconds)
df['UnsafeTimeGap'] = df['Time_Gap'] < 2

In [11]:
# Group by Vehicle_ID and compute summary statistics
agg_df = df.groupby('Vehicle_ID').agg({
    'SpeedingViolation': 'sum',
    'FastAcceleration': 'sum',
    'LaneChangeFrequency': 'max',
    'HardBraking': 'sum',
    'UnsafeHeadway': 'sum',
    'UnsafeTimeGap': 'sum',
    'Collision': 'sum'
}).reset_index()

# Rename columns for clarity
agg_df.columns = [
    'Vehicle_ID', 'SpeedingViolations', 'FastAccelerations', 
    'LaneChanges', 'HardBrakings', 'UnsafeHeadways', 
    'UnsafeTimeGaps', 'Collisions'
]

# Display the aggregated data
print(agg_df.head())

  Vehicle_ID  SpeedingViolations  FastAccelerations  LaneChanges  \
0       veh0                   8                  4            3   
1       veh1                   0                  3            0   
2      veh11                   0                  2            0   
3      veh12                   6                  3            0   
4      veh14                   5                  4            0   

   HardBrakings  UnsafeHeadways  UnsafeTimeGaps  Collisions  
0            18               0               0           0  
1            18               0               0           0  
2             6               0               0           0  
3            10               0               0           0  
4             3               0               0           0  
